In [22]:
import pandas as pd

In [23]:
df_enrolments = pd.read_csv("final_enrolment_clean.csv")
df_biometric = pd.read_csv("final_biometric_cleaned.csv")
df_demographic = pd.read_csv("df_demographic_all_final.csv")

In [24]:
df_enrolments.head()

,date,state,district,pincode,enro_age_0_5,enro_age_5_17,enro_age_18_greater
0,02-03-2025,meghalaya,east khasi hills,793121,11.0,61.0,37.0
1,09-03-2025,karnataka,bengaluru urban,560043,14.0,33.0,39.0
2,09-03-2025,uttar pradesh,kanpur nagar,208001,29.0,82.0,12.0
3,09-03-2025,uttar pradesh,aligarh,202133,62.0,29.0,15.0
4,09-03-2025,karnataka,bengaluru urban,560016,14.0,16.0,21.0


In [25]:
df_biometric.head()  


,date,pincode,bio_age_5_17,bio_age_17_,state,district
0,01-03-2025,123029,280,577,Haryana,mahendragarh
1,01-03-2025,852121,144,369,Bihar,madhepura
2,01-03-2025,185101,643,1091,Jammu And Kashmir,punch
3,01-03-2025,802158,256,980,Bihar,bhojpur
4,01-03-2025,625514,271,815,Tamil Nadu,madurai


In [26]:
df_demographic.head()

,date,pincode,demo_age_5_17,demo_age_17_,state,district
0,01-03-2025,273213,49.0,529.0,Uttar Pradesh,Gorakhpur
1,01-03-2025,517132,22.0,375.0,Andhra Pradesh,Chittoor
2,01-03-2025,360006,65.0,765.0,Gujarat,Rajkot
3,01-03-2025,532484,24.0,314.0,Andhra Pradesh,Srikakulam
4,01-03-2025,313801,45.0,785.0,Rajasthan,Udaipur


In [27]:
df_enrolments.shape, df_demographic.shape, df_biometric.shape

((1006007, 7), (1861102, 6), (1861107, 6))

In [28]:
for df in [df_enrolments, df_demographic, df_biometric]:
    df["date"] = pd.to_datetime(df["date"], errors="coerce").dt.date
    df["pincode"] = df["pincode"].astype(str).str.strip()

df_enrolments  = df_enrolments.groupby(["date","pincode"], as_index=False).sum(numeric_only=True)
df_demographic = df_demographic.groupby(["date","pincode"], as_index=False).sum(numeric_only=True)
df_biometric   = df_biometric.groupby(["date","pincode"], as_index=False).sum(numeric_only=True)

print(df_enrolments.shape, df_demographic.shape, df_biometric.shape)


(281304, 5) (524671, 4) (693662, 4)


In [29]:
merged_df = df_demographic.merge(df_biometric, on=["date","pincode"], how="outer")
merged_df = merged_df.merge(df_enrolments, on=["date","pincode"], how="outer")

print("✅ Final shape:", merged_df.shape)
merged_df.head()


✅ Final shape: (723885, 9)


,date,pincode,demo_age_5_17,demo_age_17_,bio_age_5_17,bio_age_17_,enro_age_0_5,enro_age_5_17,enro_age_18_greater
0,2025-01-03,110001,NaN,NaN,191.0,247.0,NaN,NaN,NaN
1,2025-01-03,110002,65.0,714.0,260.0,427.0,NaN,NaN,NaN
2,2025-01-03,110003,75.0,571.0,347.0,411.0,NaN,NaN,NaN
3,2025-01-03,110005,NaN,NaN,441.0,950.0,NaN,NaN,NaN
4,2025-01-03,110006,NaN,NaN,901.0,1526.0,NaN,NaN,NaN


In [32]:
print(merged_df['pincode'].unique())

['110001' '110002' '110003' ... '625023' '628229' '758085']


In [12]:
merged_df.isna().sum()

date                        0
pincode                     0
demo_age_5_17          199214
demo_age_17_           199214
bio_age_5_17            30223
bio_age_17_             30223
enro_age_0_5           442581
enro_age_5_17          442581
enro_age_18_greater    442581
dtype: int64

In [16]:
count_cols = [
    "demo_age_5_17","demo_age_17_",
    "bio_age_5_17","bio_age_17_",
    "enro_age_0_5","enro_age_5_17","enro_age_18_greater"
]

merged_df[count_cols] = merged_df[count_cols].fillna(0)


In [17]:
merged_df.isna().sum()

date                   0
pincode                0
demo_age_5_17          0
demo_age_17_           0
bio_age_5_17           0
bio_age_17_            0
enro_age_0_5           0
enro_age_5_17          0
enro_age_18_greater    0
dtype: int64

In [18]:
merged_df.shape

(723885, 9)

In [20]:
# create pincode -> state,district mapping
pincode_map = (
    df[["pincode", "state", "district"]]
    .dropna()
    .drop_duplicates(subset=["pincode"])
)

# merge mapping into merged_df
merged_df = merged_df.merge(pincode_map, on="pincode", how="left")
